In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish

llm = ChatOpenAI(temperature=0)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


In [2]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "how many letters in the word educa?"})


# intermediate_steps = []
# while True:
#     output = agent.invoke({
#         "input": "how many letters in the word educa?",
#         "intermediate_steps": intermediate_steps
#     })
#     if isinstance(output, AgentFinish):
#         final_result = output.return_values["output"]
#         break
#     else:
#         print(output.tool, output.tool_input)
#         tool = {
#             "get_word_length": get_word_length
#         }[output.tool]
#         observation = tool.run(output.tool_input)
#         intermediate_steps.append((output, observation))
# print(final_result)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}